<center>
    <img src="img/scikit-learn-logo.png" width="40%" />
    <br />
    <h1>An introduction to Machine Learning with Scikit-Learn</h1>
    <br /><br />
    Gilles Louppe (<a href="https://twitter.com/glouppe">@glouppe</a>), April 23, 2015
    <br /><br />
    <a href="https://github.com/glouppe/talk-pydata2015">https://github.com/glouppe/talk-pydata2015</a>
</center>

## About myself 

* PhD in machine learning, expertise in tree-based methods
* Core developer of Scikit-Learn
* Research Fellow working in GS-SIS, on graph and text mining problems
* Open to collaborations

## Outline

* What is machine learning?
* Scikit-Learn
* Supervised learning
* Model evaluation and selection
* Unsupervised learning

## What is machine learning?

## Scikit-Learn

- Python stack for data analysis
- API (estimator, predictors, transformers)

## Supervised learning

- Formal framework
- Linear models
- Support vector machines
- Tree-based methods

## Model evaluation and selection

- Evaluation versus selection
- Under/over-fitting
- Metrics

## Unsupervised learning

- Clustering 
- Decomposition
- Density estimation